In [ ]:
# pip install smbus2 websockets

import ssl
import json
import asyncio 
import threading
import websockets 
from smbus2 import SMBus, i2c_msg  

bus = SMBus(1) 
#bus.close() 

async def main():
    username = "hello"
    password = "iloveyou"
    hostname = "guoxiaokang.com"
    async for websocket in websockets.connect(f"wss://{username}:{password}@{hostname}:8765"): 
        try:
            await websocket.send(json.dumps({"greeting": "real_keyboard"})) 
            async for msg in websocket:
                msg   = json.loads(msg)
                if "greeting" in msg:  
                    print(msg)
                    continue
                elif msg["device_type"] == "virtual_keyboard":  
                    val   = msg["value"]
                    val_t = msg["value_type"]   
                    if val_t == "string":
                        val = ord(val)  
                    else:
                        val = int(val)
                    bus.i2c_rdwr(i2c_msg.write(0x34, [ord('k'), ord('w'), val, 255, 5])) # bytes must be in range [0, 256)
                    print("", msg) 
        except websockets.ConnectionClosed:
            print("Connection to the server is closed")
            continue 

def foo(): asyncio.run(main())
threading.Thread(target=foo).start()  